# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [15]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
# print(openai.api_key)

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [12]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2025, 8, 18)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [13]:
def get_completion(prompt, model=llm_model):
    """
    Helper function to get chat completion. 
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [14]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [6]:
# english pirate language 
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

### Ask the LLM to convert this english pirate language into American english in a clam and respectful tone 

In [7]:
style = """American English \
in a calm and respectful tone
"""

In [8]:
# prompt using the f string with the instructions as follows: 
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



### Action: run the code, change the inputs and try different variations 
### Make this work for any opensource/free model using this [langchain documentation](https://python.langchain.com/docs/introduction/) 

In [9]:
response = get_completion(prompt)

In [10]:
response

"Ah, I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

## Situation: you have different customers writing reviews in different languages. 
## Not just english pirates but also like french, german, japanise, hindi and so on! 
## Task: you have to convert all reviews in the Amarical English language. 
## Action: Use LangChain 
## Results: More managable and clean code. 

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [47]:
# langchain's abstration for the ChatGpt API endpoint 
from langchain.chat_models import ChatOpenAI

In [48]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0 whereas, the default is 0.7 
chat = ChatOpenAI(temperature=0.0, model=llm_model)

### Prompt template

In [49]:
# Let's define a template_string as follows: 
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [50]:
# To repeatedly re-use this template let's import the ChatPromptTemplate 
from langchain.prompts import ChatPromptTemplate
# create a prompt template using the template_string from_template (static method) of ChatPromptTemplate 
prompt_template = ChatPromptTemplate.from_template(template_string)


In [ ]:
# Extract the original prompt
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

### As you can see there are two input variables: 'style' and 'text' as show in the {} in the tempalte_string 

In [52]:
# prints two input variables as per prompt_templates
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [53]:
# style in which you want the customer message to be translated 
customer_style = """American English \
in a calm and respectful tone
"""

In [55]:
# Same customer email as before: 
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
# create custom_message which will generate the prompt 
# we will pass this to an LLM next to get the response 
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [56]:
# just to look at the types
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [57]:
# this is the prompt that 
# it will be creating (or you would expect this to be creating)
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [22]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

/var/folders/_9/kbclh8y12dz3_njd9xrldcm80000gp/T/ipykernel_9719/1472147048.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


In [ ]:
# check the response of the LLM
# a text translated from the english pirate to 
# polite american english 
print(customer_response.content)

I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.


### as you can imagine the other responses are in other languages, 
### and this tool can be used to translate the messages for an english 
### speaker to understand and reply to. 
### Task: try modify a prompt and see how you can get the different outputs. 

In [61]:
# now let's hope our English speaking customer service agent 
# wants to reply to the customer in their original language 
# this is a polite message 
# let's convert this into a pirate english 
# to reply to the same customer 
# in the polite tone in english pirate
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [62]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
# we are reusing the same prompt_template as we defined earlier 
# we are passing the output style we want 
# we are passing the text as a service reply 
# it should convert service reply text into polite english pirate 🤞🏴‍☠️
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)
# prompt output. 
print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [64]:
# Let's prompt to ChatGPT/LLM and print the response that it gives back
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, valued customer! Regrettably, the warranty be not coverin' the costs o' cleanin' yer galley due to yer own negligence. Ye see, 'twas yer own doin' that ye forgot to secure the lid afore startin' the blender. 'Tis a tough break, indeed! Fare thee well!


## Why do we need prompt templates? 
Ans: when we build sophisticated applications the prompts can be very detailed. (See the slide# 6). 
Prompt templates are the useful abstractions to reuse good prompt whenever you can. 

Also, LangChange provides prompts for some common operations ie. summarisation, question answering, connecting to sql dbs, connecting to various APIs, etc. by using langchain's builtin prompts and quickly get an application working without needing to engineer your own prompts. 

Another aspect of LangChain's prompt lib is that they also supports output pausing. (you will get there soon!) but when you build a complex application using LLMs, you also instruct LLMs to output the results in certain format. Such as using specific keywords. e.g. (slide #7) shows an example to carry out something called **chain-of-thought-reasoning** using a framework called the **ReAct**. 

Don't worry about the technical details here. 

- Thought: What is our LLM is thinking? It is observed that if we give space to think to an LLM it can often get to more accurate conclusion. 
- Action:  Keyword specific action (e.g. action verbs viz. search, run, bath, swim etc.)
- Observation: What to learn from the action. 

**Parser:** If you have a prompt which instructs the LLM to use these specific keywords (Thought, Action, Observation etc.), then this prompt can be coupled with a *parser* to extract out the text that has been tagged with this specific keywords. So that together provides a very nice abstraction to specify the input to an LLM and then also have a parse correctly interpret the output that the LLM provides. 

Below is an example of output parser with a LangChain: 


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [68]:
# we would like our LLM to output JSON 
# use LangChain to parse that JSON output. 

# Here the example is to extract information from a product review. 
# Format that output in a JSON format. 
# python dictionary as follows. Example of a desirable output. 
{
  "gift": False, # product is a gift? 
  "delivery_days": 5, # number of days delay 
  "price_value": "pretty affordable!" # narrative price value 
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [ ]:
# we will try to extract above type of JSON from this customer_review 
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""


# review_template takes the customer review as an input 
# and extract keys of the JSON specified above 
# Format the output as JSON with given keys 
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [70]:
# Now, let's wrap this in LangChain. 
# importing ChatPromptTemplate 
# We already did this above. 
from langchain.prompts import ChatPromptTemplate

# create a prompt_template from the review_template 
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [72]:
# create messages to pass to OpenAI 
messages = prompt_template.format_messages(text=customer_review)
# create the chat object using LangChain's OpenAI abstractor 
chat = ChatOpenAI(temperature=0.0, model=llm_model)
# invoke the LLM using messages. 
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


### Task: you should run the code till here and understand. 

In [73]:
type(response.content)

str

### most important thing to note: the output looks `JSON` but as you can see the type is `str`. 

In [33]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [34]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [74]:
# Specify what we want to parse from the LLM's response. 
# define a gift schema 
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

# define a delivery_days schema 
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

# define a price_value_schema schema 
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

# putting them all into a list
response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
# now by using this schema, langchain can give us a prompt by having the output_parser
# tell you what instructions it wants to send to the LLM 
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [37]:
format_instructions = output_parser.get_format_instructions()

In [38]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [76]:
# so here is the new review to test the parser 
# note: it also includes {format_instructions} which langchain generated above. 

review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""


# create a prompt from review_template_2 
prompt = ChatPromptTemplate.from_template(template=review_template_2)

# create the messages to be passed to the openAI endpoint 
messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [77]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [ ]:
# Call the openAI endpoint 
response = chat(messages)

In [79]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [81]:
# use output_parser that we created earlier 
output_dict = output_parser.parse(response.content)

In [44]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there"]}

In [45]:
type(output_dict)

dict

In [82]:
output_dict.get('delivery_days')

2